# Lab 8: Instruction-Based Data Generation and Classification Using Mistral-7B and Decision Trees

## Step 1: Launch a Colab Notebook and Set Up Environment Install necessary packages first: 
```bash
!pip install torch torchvision torchaudio transformers accelerate bitsandbytes 
huggingface_hub scikit-learn pandas numpy matplotlib –quiet
```

## Step 2: Load and Run Mistral Model from Hugging Face 
You can directly load Mistral-7B-Instruct-v0.1 from Hugging Face. This model supports instruction-based prompting similar to GPT-based models. 

In [ ]:
import os 
from huggingface_hub import login 

# Replace "your_token_here" with your actual Hugging Face access token 
os.environ["HUGGINGFACE_TOKEN"] = "your_token_here"  
login(token=os.environ["HUGGINGFACE_TOKEN"])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, 
BitsAndBytesConfig, pipeline 
import torch 

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1" 
quant_config = BitsAndBytesConfig(load_in_4bit=True) 
tokenizer = AutoTokenizer.from_pretrained(model_name, 
token=os.environ["HUGGINGFACE_TOKEN"]) 
model = AutoModelForCausalLM.from_pretrained( 
model_name, 
quantization_config=quant_config, 
device_map="auto", 
token=os.environ["HUGGINGFACE_TOKEN"] 
) 
text_gen_pipeline = pipeline( 
"text-generation", 
model=model, 
tokenizer=tokenizer, 
device_map="auto" 
) 

## Step 3: Run a Basic Prompt 
Here's an example to demonstrate how Mistral responds to instructions: 

In [ ]:
prompt = """ 
Generate ONLY CSV data without any explanation. It should contain exactly 30 
rows with the columns: age, income, decision (yes/no).  
Ensure that: - age values are between 18 and 65, - income ranges from 30000 to 150000, - decision has a roughly equal number of 'yes' and 'no'. 
""" 
# Increased max_new_tokens for complete CSV generation 
response = text_gen_pipeline(prompt, max_new_tokens=800) 
generated_csv = response[0]['generated_text'].strip() 
print("Generated CSV:\n", generated_csv)

## Step 4: Continue with Model Training 

In [ ]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split

In [ ]:
# Assume response is the output from text_gen_pipeline 
response_text = response[0]['generated_text'] 
 
# Split the response text into lines and then into data points 
data_points = [] 
for line in response_text.strip().split('\n'): 
    if line: 
        try: 
            age, income, purchase = line.split(',') 
            data_points.append([int(age), int(income), purchase.strip()]) 
        except ValueError: 
            # Handle lines that don't conform to the expected format 
            print(f"Skipping line: {line}") 
 
# Create a pandas DataFrame 
df = pd.DataFrame(data_points, columns=['age', 'income', 'purchase']) 
 
# Convert 'purchase' to numerical (0 for 'no', 1 for 'yes') 
df['purchase'] = df['purchase'].map({'no': 0, 'yes': 1}) 
# Define X and y 
X = df[['age', 'income']] 
y = df['purchase']

# Now you can use train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
model = DecisionTreeClassifier(max_depth=5) 
model.fit(X_train, y_train) 

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier, export_text 
from sklearn.metrics import classification_report 
# Decision rules visualization 
rules = export_text(model, feature_names=['age', 'income']) 
print("Decision Tree Rules:\n", rules) 
# Evaluate model clearly 
predictions = model.predict(X_test) 
print("Classification Report:\n", classification_report(y_test, predictions))